# Partitioning

### Objectives
1. Get partitions and cores
2. Repartition DataFrames
3. Configure default shuffle partitions

### Methods
- [DataFrame API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html): `repartition`, `coalesce`, `rdd.getNumPartitions`
- [SparkConf](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkConf.html): `get`, `set`
- [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.html): `spark.sparkContext.defaultParallelism`

### SparkConf Parameters
- `spark.sql.shuffle.partitions`
- `spark.sql.adaptive.enabled`


### Load the DataFrame from CSV

In [ ]:
# Replace with your CSV file path
csv_path = "/dbfs/tmp/events_data.csv"

df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(csv_path))

display(df)

### Get number of partitions in the DataFrame

In [ ]:
df.rdd.getNumPartitions()

### Get number of cores in the cluster

In [ ]:
print(spark.sparkContext.defaultParallelism)

SparkContext is also available as `sc`

In [ ]:
print(sc.defaultParallelism)

### Repartition DataFrame using `repartition`

In [ ]:
repartitioned_df = df.repartition(8)
repartitioned_df.rdd.getNumPartitions()

### Repartition DataFrame using `coalesce`

In [ ]:
coalesce_df = df.coalesce(8)
coalesce_df.rdd.getNumPartitions()

### Check and configure default shuffle partitions

In [ ]:
# Get current shuffle partition setting
spark.conf.get("spark.sql.shuffle.partitions")

In [ ]:
# Set shuffle partitions to match number of cores
spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)
print(spark.conf.get("spark.sql.shuffle.partitions"))

### Partitioning Guidelines
- Make the number of partitions a multiple of the number of cores
- Target a partition size of ~200MB
- Size default shuffle partitions by dividing largest shuffle stage input by ~200MB (e.g., 4TB / 200MB = 20,000)

> When writing a DataFrame to storage, the number of partitions determines the number of data files written (excluding Hive partitioning).

### Adaptive Query Execution (AQE)

In Spark 3.x, Adaptive Query Execution can dynamically coalesce shuffle partitions at runtime.

Config:
- `spark.sql.adaptive.enabled` controls AQE behavior.


In [ ]:
# Check if Adaptive Query Execution is enabled
spark.conf.get("spark.sql.adaptive.enabled")